# DX 704 Week 4 Project

This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

Submit "ratings.tsv" in Gradescope.

In [1]:
#imports
import numpy as np
import pandas as pd

#load data
recipes = pd.read_csv("recipes.tsv", sep='\t') #contains recipes
recipe_tags = pd.read_csv("recipe-tags.tsv", sep='\t') #contains tags

In [2]:
#class feature choice (following class example) -> checking if recipe has certain ingredient
def add_feature(feature_name):
    recipes[feature_name] = recipes['recipe_title'].str.lower().str.contains(feature_name.lower())
#add features
add_feature("oyster")
add_feature("mushroom")
add_feature("chicken")
add_feature("chocolate")
add_feature("almond")
add_feature("peanut")
add_feature("peach")

##### Preferences picked:
* likes: mushroom, chicken, almond, and peach
    - My thought process behind this was thinking of some of my favorite recipes. I am a big fan of any dish that has mushrooms in it. I also couldn't help but think of my dog when thinking of ingredients and she loves chicken so I threw that in too! Almond and peach came from thinking of some of my favorite desserts (almond croissants and coookies, peach cobbler, etc.)
* dislikes: oyster, chocolate, and peanut
    - I am allergic to seafood, so I looked through the "recipes.tsv" file to see if there were any seafood recipes and picked one to exclude. I am not too big of a fan of chocolate; I find it overpowering unless in small amounts, so I decided to rate it lower. I have a friend who is not a bg fan of peanut dishes, so I decided to rate this low as well!

In [3]:
#create new col.: rating (following class example)
ratings = pd.DataFrame({"recipe_title": recipes["recipe_title"],
                               "rating_raw": np.random.uniform(low=2, high=3, size=len(recipes))},
                              index=recipes.index)
#set preferences 
ratings.loc[recipes['oyster'], 'rating_raw'] = 1
ratings.loc[recipes['mushroom'], 'rating_raw'] = 5 
ratings.loc[recipes['chicken'], 'rating_raw'] = 5
ratings.loc[recipes['chocolate'], 'rating_raw'] = 1
ratings.loc[recipes['almond'], 'rating_raw'] = 5
ratings.loc[recipes['peanut'], 'rating_raw'] = 1
ratings.loc[recipes['peach'], 'rating_raw'] =5
#create rating col.
ratings['rating'] = ratings['rating_raw']
ratings.head(3)

,recipe_title,rating_raw,rating
0,Falafel,2.035218,2.035218
1,Spamburger,2.354068,2.354068
2,Bacon Fried Rice,2.533280,2.533280


In [4]:
#want ratings to be 0 to 1, with 0 = worst, 1 = best
ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25
ratings.head(3)

,recipe_title,rating_raw,rating
0,Falafel,2.035218,0.258805
1,Spamburger,2.354068,0.338517
2,Bacon Fried Rice,2.533280,0.383320


In [5]:
#check that there are at least 10 instances w/rating 1
ratings_filtered_1 = ratings.loc[ratings['rating'] == 1]
ratings_filtered_1

,recipe_title,rating_raw,rating
3,Chicken Fingers,5.0,1.0
23,Almond Chip Cookies,5.0,1.0
29,Ma La Chicken,5.0,1.0
50,Chicken Nuggets,5.0,1.0
57,Peach Crisp,5.0,1.0
62,Peach Cobbler,5.0,1.0
74,Almond Croissants,5.0,1.0
79,Mushroom Quiche,5.0,1.0
93,Bacon Wrapped Chicken,5.0,1.0
96,Chicken Alfredo Lasagna,5.0,1.0


In [6]:
#check that there are at least 10 instances w/rating 0
ratings_filtered_5 = ratings.loc[ratings['rating'] == 0]
ratings_filtered_5

,recipe_title,rating_raw,rating
6,Bacon Chocolate Chip Cookies,1.0,0.0
18,Chocolate Souffle,1.0,0.0
21,Fried Oysters,1.0,0.0
22,Peanut Butter Brownies,1.0,0.0
26,Brioche Bread With Chocolate,1.0,0.0
35,Peanut Butter Cupcakes,1.0,0.0
37,Chocolate Peanut Butter Cake,1.0,0.0
66,Chocolate Croissants,1.0,0.0
68,Chocolate Babka,1.0,0.0
77,Chocolate Cake,1.0,0.0


In [7]:
#want both 'rating' and 'recipe_slug' in tsv file:
ratings_df = pd.DataFrame({"recipe_slug": recipes["recipe_slug"],
                           "rating": ratings["rating"]})
#convert to tsv 
ratings_df.to_csv("ratings.tsv", sep="\t", index=False)

## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

In [ ]:
recipe_tags

In [ ]:
# YOUR CHANGES HERE
#already have recipe_slug col. from recipes df
#create col. bias hard-coded to one
#col. for each tag that appears in reipe-tags.tsv; 0-1 encoding of tags


Submit "features.tsv" in Gradescope.

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

In [ ]:
# YOUR CHANGES HERE

...

Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not add anything for the `intercept_` attribute of a scikit-learn model; this will be covered by the coefficient for the bias column added in part 2.

In [ ]:
# YOUR CHANGES HERE

...

Submit "model.tsv" in Gradescope.

## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [ ]:
# YOUR CHANGES HERE

...

Submit "estimates.tsv" in Gradescope.

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [ ]:
# YOUR CHANGES HERE

...

Submit "bounds.tsv" in Gradescope.

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

Hint: do not remove recipes after each recommendation.
Repeating recommendations is expected.

In [ ]:
# YOUR CHANGES HERE

...

Submit "recommendations.tsv" in Gradescope.

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.


Submit "project.ipynb" in Gradescope.